In [8]:
import sys
sys.path.append('../')
import copy
import math
import numpy as np
from utils.controller_utils import Robot
import rospy
import tools.rotations as rot
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')

%load_ext autoreload
%autoreload 2

r = Robot(optitrack_frame_names=['iiwa_base7', 'realsense_m'], position_control=True)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Kinematic model has been loaded from ../kinematics/q2pose_left.txt


[ WARN] [1743686119.241260499]: distance is not a valid solve_type; setting to default: Speed
Unknown tag "material" in /robot[@name='iiwa7']/link[@name='iiwa_link_0']/collision[1]
Unknown tag "self_collision_checking" in /robot[@name='iiwa7']/link[@name='iiwa_link_0']
Unknown tag "material" in /robot[@name='iiwa7']/link[@name='iiwa_link_1']/collision[1]
Unknown tag "material" in /robot[@name='iiwa7']/link[@name='iiwa_link_2']/collision[1]
Unknown tag "material" in /robot[@name='iiwa7']/link[@name='iiwa_link_3']/collision[1]
Unknown tag "material" in /robot[@name='iiwa7']/link[@name='iiwa_link_4']/collision[1]
Unknown tag "material" in /robot[@name='iiwa7']/link[@name='iiwa_link_5']/collision[1]
Unknown tag "material" in /robot[@name='iiwa7']/link[@name='iiwa_link_6']/collision[1]
Unknown tag "material" in /robot[@name='iiwa7']/link[@name='iiwa_link_7']/collision[1]
Unknown tag "hardwareInterface" in /robot[@name='iiwa7']/transmission[@name='iiwa_tran_1']/actuator[@name='iiwa_motor_1']

In [23]:
r.qh

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
#pre-defined pose
home_pose = np.array([ 0.03063398, -0.15441399,  0.82104033,  0.82512679,  0.00664029,
       -0.2039733 ,  0.80409743,  0.        , -0.10271061, -0.06070025,
        0.90650362,  0.81377007,  0.86300686,  0.44554398,  0.08874171,
        0.77574977])

catch_pose = np.array([-0.50889807,  1.74874918,  0.12737114,  0.39226041,  0.07008444,
        1.64808905,  1.36754356,  0.21940653,  0.0937576 ,  1.77053021,
        1.36580765,  0.55991777,  0.94150295,  0.14131779, -0.24060529,
        0.96755097])

envelop_pose = np.array([ 0.07941064,  1.77857954,  0.44877399,  0.62444235,  0.29191241,
        1.55038098,  0.61655537,  0.36046541,  0.4311105 ,  1.76264807,
        1.02352552,  0.9226552 ,  1.54144452,  0.48900141, -0.08175917,
        1.3566044 ])

while True:
    r.move_to_joints(home_pose,vel=[0.2,8.0])
    rospy.sleep(2)
    r.move_to_joints(envelop_pose, vel=[0.2,8.0])
    rospy.sleep(2)

In [ ]:
# position control
r.iiwa_go_home()

In [ ]:
# impedance control
target_pose = r.x
target_pose[0] -=0.2
target_pose[2] -=0.2
efforts = []
r.iiwa_cartesion_impedance_control(target_pose,vel=1.0)


In [ ]:
# move along a circle under impedance control
def generate_circle_via_points(current_pose, radius=0.05, num_points=10):
    x0, y0, z0 = current_pose[:3]
    center_x = x0 - radius
    center_y = y0
    center_z = z0

    theta_array = np.linspace(0, 2*math.pi, num_points)

    via_points = np.zeros((num_points, 7))
    for i, theta in enumerate(theta_array):
        # x = center_x + radius*cos(theta), y = center_y + radius*sin(theta)
        x = center_x + radius * math.cos(theta)
        y = center_y + radius * math.sin(theta)
        z = center_z
        via_points[i, :3] = np.array([x, y, z])

        via_points[i, 3:] = current_pose[3:]
    return via_points

current_pose = r.x
via_points = generate_circle_via_points(current_pose, radius=0.2, num_points=20)

r.iiwa_cartesion_impedance_control(via_points)

In [ ]:
target_pose = np.copy(r.x)
target_pose[2] -= 0.2
q = r.trac_ik_solver(target_pose)
# r.move_to_joints(q, vel=[0.1, 1.0])
r.move_to_target_cartesian_pose(target_pose)
print(r.q - q)

In [ ]:
import sys
sys.path.append("../")
import utils.mujoco_interface
import mujoco
from mujoco import viewer

xml_path = '../description/iiwa7_allegro_ycb.xml'
obj_name = ''
model = mujoco.MjModel.from_xml_path(xml_path)
data = mujoco.MjData(model)
mujoco.mj_step(model, data)

# viewer.launch(model, data)
view = viewer.launch(model, data)
# in notebook, we need ro run view.sync() manually, and set auto_sync=False

obj_names = ['banana', 'bottle', 'chip_can', 'soft_scrub', 'sugar_box']
num = 0
obj = obj_names[num]
r = mujoco_interface.Robot(model, data, view, auto_sync=False, obj_names=obj_names)

q0 = np.array([-0.32032434,  0.02706913, -0.22881953, -1.42621454,  1.3862661 , 0.55966738,  1.79477984 - np.pi*3/2])
r.d.qpos[:7]  = q0

r.step()
view.sync()


In [ ]:
r.modify_joint(q0)
# move the object to a proper pose
p = np.array([0.59,-0.315,1.3])
q = rot.euler2quat([np.pi/2,np.pi,np.pi/18*1.5])
pose = np.concatenate([p, q])
r.modify_obj_pose(obj, pose)

# move the hand to the grasping configuration

qh = np.array([0, 0.33, 1.05, 1.05,
               0, 0.29, 1.2, 0.6,
               0, 0.32, 1.2, 0.8,
               1, 0.5, 0.71,1])
r.modify_joint(qh)

r.step()
view.sync()

In [ ]:
import gym
import torch
from stable_baselines3 import PPO, DDPG, TD3, SAC, HerReplayBuffer

log_dir = './panda_reach_v2_tensorboard/'
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

env = gym.make('HandReach')
model = PPO('MultiInputPolicy', env,
            verbose=1,
            device=device)
model.learn(total_timesteps=100000)
model.save("ppo_panda_reach_v2")



In [ ]:
obs = env.reset()
for _ in range(1000):
    action, _states = model.predict(obs,deterministic=True)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done:
        obs = env.reset()